# Datavalanche


The goal of our project is to highlight the causes of avalanches which kill around 30 people per year in Switzerland. To do so we plan to use public data on mortal avalanche accidents available on the Schnee- und Lawinenforschung's website, we will also use open weather data in Switzerland.

The notebook will be structured as follow:

    1) Data Wrangling and Exploratory Data Analysis
    2) Map Vizualisations
    3) Causes of avalanches 

## 1) Data Wrangling and Exploratory Data Analysis

### Exploratory data analysis of the SLF deadly avalanche dataset

In [ ]:
# import useful librairies

%load_ext autoreload
%autoreload 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import date
import folium
from folium import plugins
%aimport helpers
import calendar
import datetime
import seaborn as sns
#import pygrib

The data that is used below (`data/avalanches.csv`) is the data that is published at http://www.slf.ch/praevention/lawinenunfaelle/unfaelle_langj/index_EN

It is obtained through the script `avalanche_data_processor.js`

In [ ]:
# Read the slf data
data = pd.read_csv('data/avalanches.csv', parse_dates=['date_posix_ts'], date_parser=helpers.parsedate)
data.head()

Let's see how many deadly avalanches occurred each year during the last 20 years

In [ ]:
avalanches_per_winter = data.groupby('winter').count()['canton']
fig = plt.figure()
fig.suptitle('Avalanches per winter', fontsize=14, fontweight='bold')
avalanches_per_winter.plot(kind='bar')

In [ ]:
# Plot avalanches per month
data['month'] = data['date_posix_ts'].map(lambda x: x.month)
avalanches_per_month = data.groupby('month').count()['canton']
fig = plt.figure()
fig.suptitle('Avalanches per month', fontsize=14, fontweight='bold')
avalanches_per_month.plot(kind='bar')

In [ ]:
# Show the number of avalanches per month per winter
avalanches_per_winter_and_month = data.groupby(['winter', 'month']).count()[['canton']]
avalanches_per_winter_and_month.columns = ['count']
avalanches_per_winter_and_month.head(18)



Let's now look at the aspect of the slopes where the avalanches occurred.

In [ ]:
avalanches_per_aspect = data.groupby('aspect_string').count()['canton']
fig = plt.figure()
fig.suptitle('Avalanches per slope´s orientation' , fontsize=14, fontweight='bold')
avalanches_per_aspect.plot(kind='bar')

It seems that north facing slopes are more dangerous. 

In [ ]:
# The danger level is define as follow: http://www.slf.ch/schneeinfo/zusatzinfos/lawinenskala-europa/index_EN
avalanches_per_danger = data.groupby('danger_level').count()['canton']
fig = plt.figure()
fig.suptitle('Avalanches per danger level', fontsize=14, fontweight='bold')
avalanches_per_danger.plot(kind='bar')

Let's see the distribution of avalanche starting point elevation

In [ ]:
print('Mean avalanche start point elevation:', data['elevation'].mean())
fig = plt.figure()
fig.suptitle('Avalanches per elevation', fontsize=14, fontweight='bold')
data['elevation'].hist(bins=20)

### Observations and Remarks

It is important to note that the avalanches that are present in the dataset are the ones that have caught at least one person.
We can see that on average every winter there are about 18 avalanches which are recorded.
These avalanches occurs mainly between Decemember and April, whith February the worst month, which recorded more than 100 avalanches these last 20 years. We can also see that most avalanches occur on faces that are facing North (N, NE and NW). 
The distribution of the avalanches per elevation looks like a Guassian with mean 2500. Most of the time the danger level is 3 when an avalanch occurs, the fact there are less avalanch with higher risk probably due to the fact that these risks happen less frequently and also people will be more inclined to not go in dangerous places when the risk is high enough. 



## 2) Map Vizualisations

In this second part of the notebook we will do some vizualisations about the data on a map
Note that the markers on the map allow you to view the geographic location in 3D, but it seems like the feature doesn't work quite well.

In [ ]:
# Get number of avalanches per canton
missing_cantons = ['ZH', 'ZG', 'SO', 'BS', 'BL', 'SH', 'JU', 'GE', 'NE','TG', 'AR', 'AG']
avalanches_per_canton = data.groupby('canton').count()[['winter']]
avalanches_per_canton.columns = [['count']]
# Apply a log scale
avalanches_per_canton['count'] = avalanches_per_canton['count'].apply(lambda x: 0 if x == 0 else np.log10(x))
avalanches_per_canton = avalanches_per_canton.append(pd.DataFrame({'count':0}, index=missing_cantons))
avalanches_per_canton.reset_index(inplace=True)

In [ ]:
# Change the stupid coordinate system in latitude/longitude
for i in data.index:
    x = data.starting_zone_X[i]
    y = data.starting_zone_Y[i]
    lat = helpers.CHtoWGSlat(x, y)
    lng = helpers.CHtoWGSlng(x, y)
    data.set_value(i, 'lat', lat)
    data.set_value(i, 'lon', lng)
    
data.drop(['starting_zone_X','starting_zone_Y'], axis=1, inplace=True)

In [ ]:
# Count the number of avalanches per community
d2 = data[['community', 'lat', 'lon']]
cnt = d2.groupby('community').count()[['lat']]
cnt.columns = ['count']
avalanches_per_community = pd.concat([d2.groupby('community').first(), cnt], axis=1)
avalanches_per_community.head()

In [ ]:
topo_path = r'ch-cantons.topojson.json'
# Create map with log number of avalanches per canton 
av_per_canton_map = folium.Map(location=[46.8, 8.239], zoom_start=8)
av_per_canton_map.choropleth(geo_path = topo_path, data=avalanches_per_canton,
                     columns=['canton', 'count'], 
                     key_on='feature.id',
                     fill_color='PuRd', fill_opacity=0.7, line_opacity=0.2,
                     threshold_scale=[0, 0.5, 1, 1.5, 2, 2.5], 
                     topojson='objects.cantons')
helpers.addMarker(avalanches_per_community, av_per_canton_map, 30)

__Avalanches per canton__

The markers are 30 places that had the most number of deadly avalanches the last 20 years 

In [ ]:
av_per_canton_map

In [ ]:
# Create a heat_map of avalanches
avalanches_heat_map = folium.Map(location=[46.8, 8.239], zoom_start=8, tiles='Mapbox Bright', )
avalanches_heat_map.choropleth(geo_path = topo_path, topojson='objects.cantons', fill_opacity=0.3, line_opacity=0.7), 
avalanches_heat_map_vals = avalanches_per_community.as_matrix()
avalanches_heat_map.add_children(plugins.HeatMap(avalanches_heat_map_vals, radius = 17))
helpers.addMarker(avalanches_per_community, avalanches_heat_map)


__Avalanches heat map__

The markers correspond to the 10 places with most number of avalanches

In [ ]:
avalanches_heat_map

### Observations and Remarks

Most of the avalanches happen in the Bern, Valais and Grison canton. This is not too surprising since the Alpes are situated in those canton.
From those three canton the highest density of avalanches happens in Valais.

## Meteorological data

Read the meteorological data for the 10 places with the most avalanches

In [ ]:
#read the meteorological data
meteo_data = pd.read_pickle('./data/meteo_data.pkl')
#fix the index
meteo_data = meteo_data.set_index([list(range(0, meteo_data.shape[0]))])

### Some data exploration

Let's look at a few of the features monthly and by winter

In [ ]:
nb_places = meteo_data.groupby(['Latitude', 'Longitude']).mean().shape[0]
#since we're going to average over everything, we might as well drop the coordinates
meteo_data_monthly = meteo_data.copy().drop(['Latitude', 'Longitude'], axis=1)
#get rid of the day
meteo_data_monthly['Date'] = meteo_data_monthly['Date'].map(lambda x: datetime.datetime(x.year, x.month, 1))
#group by month, averaging most of the parameters but summing the snowfall and sunshine duration,
#and averaging them by locations
agg_functions = {}
agg_functions.update(dict.fromkeys(meteo_data_monthly.columns.drop(['Date', 'Sunshine duration', 'Snowfall']), 'mean'))
agg_functions.update(dict.fromkeys(['Sunshine duration', 'Snowfall'], lambda x: x.sum()/nb_places))
meteo_data_monthly = meteo_data_monthly.groupby('Date', as_index=False).agg(agg_functions)

In [ ]:
#replace the date by the month and the winter, respectively
meteo_data_month = meteo_data_monthly.copy()
#keep only the month
meteo_data_month['Date'] = meteo_data_month['Date'].map(lambda x: x.month)
#sort by month
meteo_data_month.sort_values(by='Date', inplace=True)
#replace the month by its name
meteo_data_month['Date'] = meteo_data_month['Date'].map(lambda x: calendar.month_name[x])
#group by month
meteo_data_month = meteo_data_month.groupby('Date', sort=False).mean()

meteo_data_winter = meteo_data_monthly.copy()
#replace the date by the winter it belongs to
meteo_data_winter['Date'] = meteo_data_winter['Date'].map(lambda x: helpers.date_to_winter(x))
#group by winter and drop the first and the last since the data isn't complete
meteo_data_winter = meteo_data_winter.groupby('Date').agg(agg_functions).drop(['1996-1997', '2016-2017'])

Let's plot the average snow depth, first by month then by winter

In [ ]:
sns.set_style('darkgrid')
meteo_data_month.plot(y='Snow depth', yticks=np.arange(0, 1.1, 0.1), kind='bar')

In [ ]:
ax = meteo_data_winter.plot(y='Snow depth', yticks=np.arange(0, 0.51, 0.1))

ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)

In [ ]:
meteo_data_month.plot(y='Sunshine duration', yticks=range(0, 201, 10), kind='bar')


No big surprise there. At least the data seems to make some sort of sense.

In [ ]:
ax = meteo_data_winter.plot(y='Sunshine duration', yticks=range(194, 211, 1))
ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)

[Temperature](https://www.youtube.com/watch?v=dW2MmuA1nI4)

In [ ]:
meteo_data_month.plot(y='2 metre temperature', yticks=range(-5, 15), kind='bar')

And for the winters.

In [ ]:
ax = meteo_data_winter.plot(y='2 metre temperature', yticks=np.arange(4, 7.1, 0.25))
ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)

## Machine learning

Now that we know a little more about our data, let's try to see if we can predict avalanches. First we need to prepare our data a little bit more: add a column saying if there was an avalanche that date or not, drop the useless columns (date and coordinates).

In [ ]:
data_with_ts = data.copy().drop(['starting_zone_X', 'starting_zone_Y'], axis=1)
data_with_ts['date_posix_ts'] = data_with_ts['date_posix_ts'].map(lambda date: date.timestamp())

In [ ]:
row = meteo_data.loc[0]

In [ ]:
data_with_ts[(abs(data_with_ts['lat']-row['Latitude']) < 1e-2) 
         & (abs(data_with_ts['lon']-row['Longitude']) < 1e-2)]['date_posix_ts'].values[0]

In [ ]:
def f(a):
    lat = float(a.split('/')[1])+5e-4
    lon = float(a.split('/')[0])+5e-4
    return data_with_ts[(abs(data_with_ts['lat']-lat) < 5e-3) 
         & (abs(data_with_ts['lon']-lon) < 5e-3)]

In [ ]:
f('7.939/47.004')

### def find_avalanche(row):
    avalanche = data_with_ts[(abs(data_with_ts['lat']-row['Latitude']) < 5e-3) 
         & (abs(data_with_ts['lon']-row['Longitude']) < 5e-3)]
    return pd.Series({'Avalanche': int((avalanche['date_posix_ts'] == row['Date'].timestamp()).values[0]),
                         'Elevation': avalanche['elevation'].values[0],
                         'Orientation': avalanche['aspect_string'].values[0]})

In [ ]:
meteo_data[['Avalanche', 'Elevation', 'Orientation']] = meteo_data.apply(find_avalanche, axis=1)

In [ ]:
meteo_data.shape